# Imports and utility functions

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import matplotlib.pyplot as plt # plotting
import matplotlib.image as mpimg # images
import numpy as np #numpy
import seaborn as sns
import tensorflow.compat.v2 as tf #use tensorflow v2 as a main 
import tensorflow.keras as keras # required for high level applications
from sklearn.model_selection import train_test_split # split for validation sets
from sklearn.metrics import accuracy_score, f1_score,classification_report
import scipy
import pandas as pd
import re

In [ ]:
def cleanTexts(texts):
    cleaned = []
    pattern = "[^a-zA-Z0-9]"
    for text in texts:
        clrd = re.sub(pattern," ",text).lower().strip()
        cleaned.append(clrd)
    return cleaned

# Load data

In [ ]:
from sklearn.utils import shuffle

dataset = pd.read_csv('train_data_tweetsENG.csv')
dataset = shuffle(dataset)
dataset.tail()

,text,label
79206,"@justEdith Ah, Shuckle berries, eye think you'...",1
312574,nothing beats being vomited on at 2am by your ...,0
630603,Washing windows today,1
498472,"First day off this week,and it's already over!...",0
495358,"@RetroRewind yes, Games first. Then can we fol...",1


In [ ]:
x_train = list(cleanTexts(dataset['text']))
# print(x[:5])

y_train = list(dataset['label'])
# print(y[:5])

#x_train = x_train[:20000]
#y_train = y_train[:20000]

#x_train = x_train[:500000]
#y_train = y_train[:500000]

x_train = x_train[:900000]
y_train = y_train[:900000]

In [ ]:
test_data = pd.read_csv('test_data_tweetsENG.csv')

x_test = list(cleanTexts(test_data['text']))
y_test = list(test_data['label'])

x_test = x_test[:10000]
y_test = y_test[:10000]

# Train model and predict on test dataset

In [ ]:
from tensorflow import string as tf_string
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.compat.v1.keras.layers import CuDNNGRU, CuDNNLSTM
from tensorflow.keras.layers import LSTM, GRU, Bidirectional

# Import FastText embeddings

In [ ]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.vec.gz

--2022-04-17 14:24:43--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.vec.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.75.142, 104.22.74.142, 172.67.9.4, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.75.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1325960915 (1.2G) [binary/octet-stream]
Saving to: ‘cc.en.300.vec.gz’

cc.en.300.vec.gz    100%[===================>]   1.23G  56.1MB/s    in 24s     

2022-04-17 14:25:07 (52.5 MB/s) - ‘cc.en.300.vec.gz’ saved [1325960915/1325960915]



In [ ]:
import gzip

In [ ]:
!gzip -d cc.en.300.vec.gz

In [ ]:
path_to_fasttext_file = './cc.en.300.vec'

embeddings_index = {}
with open(path_to_fasttext_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

Found 2000000 word vectors.


# 1 - 6 Vectorizer parameters

In [ ]:
from tensorflow import string as tf_string
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

embedding_dim = 300 # Dimension of embedded representation 
vocab_size = 30000 # Number of unique tokens in vocabulary
sequence_length = 30 # Output dimension after vectorizing

vect_layer = TextVectorization(max_tokens=vocab_size, output_mode='int', output_sequence_length=sequence_length)
vect_layer.adapt(x_train)

In [ ]:
voc = vect_layer.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))

In [ ]:
num_tokens = len(voc) + 2
hits = 0
misses = 0


embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

Converted 25375 words (4625 misses)


In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.10, random_state=69, stratify=y_train)

## 1.Experiment

In [ ]:
input_layer = keras.layers.Input(shape=(1,), dtype=tf_string)
x_v = vect_layer(input_layer)
emb = keras.layers.Embedding(num_tokens, embedding_dim, embeddings_initializer=keras.initializers.Constant(embedding_matrix), trainable=False)(x_v)
x = LSTM(64, activation='relu', return_sequences=True)(emb)
x = GRU(64, activation='relu', return_sequences=False)(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(64, 'relu')(x)
x = keras.layers.Dense(32, 'relu')(x)
x = keras.layers.Dropout(0.2)(x)
output_layer = keras.layers.Dense(1, 'sigmoid')(x)

model = keras.Model(input_layer, output_layer)
model.summary()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

model.compile(optimizer=optimizer, loss=keras.losses.BinaryCrossentropy(), metrics=['accuracy'])

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 30)               0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 30, 300)           9000600   
                                                                 
 lstm (LSTM)                 (None, 30, 64)            93440     
                                                                 
 gru (GRU)                   (None, 64)                24960     
                                                                 
 flatten (Flatten)           (None, 64)                0         
                                                             

In [ ]:
es = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=70, restore_best_weights=True)

batch_size = 768
epochs = 5
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), callbacks=[es], epochs=epochs, batch_size=batch_size)

Epoch 1/5
24/24 [==============================] - 9s 156ms/step - loss: 0.6900 - accuracy: 0.5272 - val_loss: 0.6883 - val_accuracy: 0.5425
Epoch 2/5
24/24 [==============================] - 3s 126ms/step - loss: 0.6797 - accuracy: 0.5713 - val_loss: 0.6479 - val_accuracy: 0.6500
Epoch 3/5
24/24 [==============================] - 3s 123ms/step - loss: 0.6161 - accuracy: 0.6932 - val_loss: 0.5572 - val_accuracy: 0.7320
Epoch 4/5
24/24 [==============================] - 3s 127ms/step - loss: 0.5397 - accuracy: 0.7424 - val_loss: 0.5113 - val_accuracy: 0.7665
Epoch 5/5
24/24 [==============================] - 3s 127ms/step - loss: 0.5049 - accuracy: 0.7582 - val_loss: 0.5031 - val_accuracy: 0.7715


In [ ]:
model.evaluate(x_test,y_test)

313/313 [==============================] - 4s 14ms/step - loss: 0.5171 - accuracy: 0.7456


[0.5170741081237793, 0.7455999851226807]

In [ ]:
y_pred=model.predict(x_test)

accuracy_sc = accuracy_score(y_pred=y_pred.round(),y_true=y_test)*100
f1_sc = f1_score(y_pred=y_pred.round(),y_true=y_test)

print("Accuracy score is {}% ".format(accuracy_sc))
print("f1-score is {}% ".format(f1_sc))
print(classification_report(y_pred=y_pred.round(),y_true=y_test))

Accuracy score is 74.56% 
f1-score is 0.7228154281978644% 
              precision    recall  f1-score   support

           0       0.71      0.83      0.76      5000
           1       0.79      0.66      0.72      5000

    accuracy                           0.75     10000
   macro avg       0.75      0.75      0.74     10000
weighted avg       0.75      0.75      0.74     10000



## 2.Experiment

In [ ]:
input_layer = keras.layers.Input(shape=(1,), dtype=tf_string)
x_v = vect_layer(input_layer)
emb = keras.layers.Embedding(num_tokens, embedding_dim, embeddings_initializer=keras.initializers.Constant(embedding_matrix), trainable=False)(x_v)
x = Bidirectional(LSTM(64, activation='relu', return_sequences=True))(emb)
x = Bidirectional(GRU(64, activation='relu', return_sequences=False))(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(64, 'relu')(x)
x = keras.layers.Dense(32, 'relu')(x)
x = keras.layers.Dropout(0.2)(x)
output_layer = keras.layers.Dense(1, 'sigmoid')(x)

model = keras.Model(input_layer, output_layer)
model.summary()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

model.compile(optimizer=optimizer, loss=keras.losses.BinaryCrossentropy(), metrics=['accuracy'])

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 30)               0         
 torization)                                                     
                                                                 
 embedding_2 (Embedding)     (None, 30, 300)           9000600   
                                                                 
 bidirectional_2 (Bidirectio  (None, 30, 128)          186880    
 nal)                                                            
                                                                 
 bidirectional_3 (Bidirectio  (None, 128)              74496     
 nal)                                                            
                                                           

In [ ]:
es = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=70, restore_best_weights=True)

batch_size = 128
epochs = 3
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), callbacks=[es], epochs=epochs, batch_size=batch_size)

Epoch 1/3
141/141 [==============================] - 43s 275ms/step - loss: 0.6104 - accuracy: 0.6612 - val_loss: 0.5251 - val_accuracy: 0.7565
Epoch 2/3
141/141 [==============================] - 32s 226ms/step - loss: 0.5128 - accuracy: 0.7553 - val_loss: 0.5055 - val_accuracy: 0.7695
Epoch 3/3
141/141 [==============================] - 31s 223ms/step - loss: 0.4794 - accuracy: 0.7755 - val_loss: 0.4958 - val_accuracy: 0.7795


In [ ]:
model.evaluate(x_test,y_test)

313/313 [==============================] - 6s 20ms/step - loss: 0.5012 - accuracy: 0.7639


[0.5012386441230774, 0.7638999819755554]

In [ ]:
y_pred=model.predict(x_test)

accuracy_sc = accuracy_score(y_pred=y_pred.round(),y_true=y_test)*100
f1_sc = f1_score(y_pred=y_pred.round(),y_true=y_test)

print("Accuracy score is {}% ".format(accuracy_sc))
print("f1-score is {}% ".format(f1_sc))
print(classification_report(y_pred=y_pred.round(),y_true=y_test))

Accuracy score is 76.39% 
f1-score is 0.7518133081046987% 
              precision    recall  f1-score   support

           0       0.74      0.81      0.77      5000
           1       0.79      0.72      0.75      5000

    accuracy                           0.76     10000
   macro avg       0.77      0.76      0.76     10000
weighted avg       0.77      0.76      0.76     10000



## 3.Experiment

In [ ]:
input_layer = keras.layers.Input(shape=(1,), dtype=tf_string)
x_v = vect_layer(input_layer)
emb = keras.layers.Embedding(num_tokens, embedding_dim, embeddings_initializer=keras.initializers.Constant(embedding_matrix), trainable=False)(x_v)
x = LSTM(64, activation='relu', return_sequences=True)(emb)
x = GRU(64, activation='relu', return_sequences=False)(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(64, 'relu')(x)
x = keras.layers.Dense(32, 'relu')(x)
x = keras.layers.Dropout(0.2)(x)
output_layer = keras.layers.Dense(1, 'sigmoid')(x)

model = keras.Model(input_layer, output_layer)
model.summary()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

model.compile(optimizer=optimizer, loss=keras.losses.BinaryCrossentropy(), metrics=['accuracy'])

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 30)               0         
 ectorization)                                                   
                                                                 
 embedding_3 (Embedding)     (None, 30, 300)           9000600   
                                                                 
 lstm_3 (LSTM)               (None, 30, 64)            93440     
                                                                 
 gru_3 (GRU)                 (None, 64)                24960     
                                                                 
 flatten_3 (Flatten)         (None, 64)                0         
                                                           

In [ ]:
es = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=70, restore_best_weights=True)

batch_size = 768
epochs = 5
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), callbacks=[es], epochs=epochs, batch_size=batch_size)

Epoch 1/5
586/586 [==============================] - 78s 129ms/step - loss: 0.5007 - accuracy: 0.7526 - val_loss: 0.4482 - val_accuracy: 0.7912
Epoch 2/5
586/586 [==============================] - 75s 128ms/step - loss: 0.4388 - accuracy: 0.7965 - val_loss: 0.4335 - val_accuracy: 0.7967
Epoch 3/5
586/586 [==============================] - 75s 129ms/step - loss: 0.4208 - accuracy: 0.8072 - val_loss: 0.4198 - val_accuracy: 0.8077
Epoch 4/5
586/586 [==============================] - 75s 127ms/step - loss: 0.4102 - accuracy: 0.8132 - val_loss: 0.4098 - val_accuracy: 0.8114
Epoch 5/5
586/586 [==============================] - 78s 133ms/step - loss: 0.4028 - accuracy: 0.8171 - val_loss: 0.4088 - val_accuracy: 0.8131


In [ ]:
model.evaluate(x_test,y_test)

313/313 [==============================] - 4s 12ms/step - loss: 0.4103 - accuracy: 0.8117


[0.4102845788002014, 0.8116999864578247]

In [ ]:
y_pred=model.predict(x_test)

accuracy_sc = accuracy_score(y_pred=y_pred.round(),y_true=y_test)*100
f1_sc = f1_score(y_pred=y_pred.round(),y_true=y_test)

print("Accuracy score is {}% ".format(accuracy_sc))
print("f1-score is {}% ".format(f1_sc))
print(classification_report(y_pred=y_pred.round(),y_true=y_test))

Accuracy score is 81.17% 
f1-score is 0.8156632403328439% 
              precision    recall  f1-score   support

           0       0.83      0.79      0.81      5000
           1       0.80      0.83      0.82      5000

    accuracy                           0.81     10000
   macro avg       0.81      0.81      0.81     10000
weighted avg       0.81      0.81      0.81     10000



## 4.Experiment

In [ ]:
input_layer = keras.layers.Input(shape=(1,), dtype=tf_string)
x_v = vect_layer(input_layer)
emb = keras.layers.Embedding(num_tokens, embedding_dim, embeddings_initializer=keras.initializers.Constant(embedding_matrix), trainable=False)(x_v)
x = Bidirectional(LSTM(64, activation='relu', return_sequences=True))(emb)
x = Bidirectional(GRU(64, activation='relu', return_sequences=False))(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(64, 'relu')(x)
x = keras.layers.Dense(32, 'relu')(x)
x = keras.layers.Dropout(0.2)(x)
output_layer = keras.layers.Dense(1, 'sigmoid')(x)

model = keras.Model(input_layer, output_layer)
model.summary()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

model.compile(optimizer=optimizer, loss=keras.losses.BinaryCrossentropy(), metrics=['accuracy'])

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 30)               0         
 ectorization)                                                   
                                                                 
 embedding_4 (Embedding)     (None, 30, 300)           9000600   
                                                                 
 bidirectional_4 (Bidirectio  (None, 30, 128)          186880    
 nal)                                                            
                                                                 
 bidirectional_5 (Bidirectio  (None, 128)              74496     
 nal)                                                            
                                                           

In [ ]:
es = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=70, restore_best_weights=True)

batch_size = 768
epochs = 4 
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), callbacks=[es], epochs=epochs, batch_size=batch_size)

In [ ]:
model.evaluate(x_test,y_test)

313/313 [==============================] - 12s 39ms/step - loss: 0.4055 - accuracy: 0.8127


[0.4054867923259735, 0.8126999735832214]

In [ ]:
y_pred=model.predict(x_test)

accuracy_sc = accuracy_score(y_pred=y_pred.round(),y_true=y_test)*100
f1_sc = f1_score(y_pred=y_pred.round(),y_true=y_test)

print("Accuracy score is {}% ".format(accuracy_sc))
print("f1-score is {}% ".format(f1_sc))
print(classification_report(y_pred=y_pred.round(),y_true=y_test))

Accuracy score is 81.27% 
f1-score is 0.8181023599106536% 
              precision    recall  f1-score   support

           0       0.83      0.78      0.81      5000
           1       0.80      0.84      0.82      5000

    accuracy                           0.81     10000
   macro avg       0.81      0.81      0.81     10000
weighted avg       0.81      0.81      0.81     10000



## 5.Experiment

In [ ]:
input_layer = keras.layers.Input(shape=(1,), dtype=tf_string)
x_v = vect_layer(input_layer)
emb = keras.layers.Embedding(num_tokens, embedding_dim, embeddings_initializer=keras.initializers.Constant(embedding_matrix), trainable=False)(x_v)
x = LSTM(64, activation='relu', return_sequences=True)(emb)
x = GRU(64, activation='relu', return_sequences=False)(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(64, 'relu')(x)
x = keras.layers.Dense(32, 'relu')(x)
x = keras.layers.Dropout(0.2)(x)
output_layer = keras.layers.Dense(1, 'sigmoid')(x)

model = keras.Model(input_layer, output_layer)
model.summary()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

model.compile(optimizer=optimizer, loss=keras.losses.BinaryCrossentropy(), metrics=['accuracy'])

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_2 (TextV  (None, 30)               0         
 ectorization)                                                   
                                                                 
 embedding_5 (Embedding)     (None, 30, 300)           9000600   
                                                                 
 lstm_5 (LSTM)               (None, 30, 64)            93440     
                                                                 
 gru_5 (GRU)                 (None, 64)                24960     
                                                                 
 flatten_5 (Flatten)         (None, 64)                0         
                                                           

In [ ]:
es = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=70, restore_best_weights=True)

batch_size = 768
epochs = 5
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), callbacks=[es], epochs=epochs, batch_size=batch_size)

Epoch 1/5
1055/1055 [==============================] - 146s 135ms/step - loss: 0.4725 - accuracy: 0.7740 - val_loss: 0.4279 - val_accuracy: 0.8028
Epoch 2/5
1055/1055 [==============================] - 141s 134ms/step - loss: 0.4218 - accuracy: 0.8068 - val_loss: 0.4111 - val_accuracy: 0.8117
Epoch 3/5
1055/1055 [==============================] - 142s 134ms/step - loss: 0.4080 - accuracy: 0.8148 - val_loss: 0.3988 - val_accuracy: 0.8186
Epoch 4/5
1055/1055 [==============================] - 140s 133ms/step - loss: 0.3988 - accuracy: 0.8196 - val_loss: 0.3934 - val_accuracy: 0.8219
Epoch 5/5
1055/1055 [==============================] - 145s 138ms/step - loss: 0.3927 - accuracy: 0.8231 - val_loss: 0.3952 - val_accuracy: 0.8191


In [ ]:
model.evaluate(x_test,y_test)

313/313 [==============================] - 4s 12ms/step - loss: 0.3990 - accuracy: 0.8177


[0.399008184671402, 0.8177000284194946]

In [ ]:
y_pred=model.predict(x_test)

accuracy_sc = accuracy_score(y_pred=y_pred.round(),y_true=y_test)*100
f1_sc = f1_score(y_pred=y_pred.round(),y_true=y_test)

print("Accuracy score is {}% ".format(accuracy_sc))
print("f1-score is {}% ".format(f1_sc))
print(classification_report(y_pred=y_pred.round(),y_true=y_test))

Accuracy score is 81.77% 
f1-score is 0.824829441721918% 
              precision    recall  f1-score   support

           0       0.85      0.78      0.81      5000
           1       0.79      0.86      0.82      5000

    accuracy                           0.82     10000
   macro avg       0.82      0.82      0.82     10000
weighted avg       0.82      0.82      0.82     10000



## 6.Experiment

In [ ]:
input_layer = keras.layers.Input(shape=(1,), dtype=tf_string)
x_v = vect_layer(input_layer)
emb = keras.layers.Embedding(num_tokens, embedding_dim, embeddings_initializer=keras.initializers.Constant(embedding_matrix), trainable=False)(x_v)
x = Bidirectional(LSTM(64, activation='relu', return_sequences=True))(emb)
x = Bidirectional(GRU(64, activation='relu', return_sequences=False))(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(64, 'relu')(x)
x = keras.layers.Dense(32, 'relu')(x)
x = keras.layers.Dropout(0.2)(x)
output_layer = keras.layers.Dense(1, 'sigmoid')(x)

model = keras.Model(input_layer, output_layer)
model.summary()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

model.compile(optimizer=optimizer, loss=keras.losses.BinaryCrossentropy(), metrics=['accuracy'])

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_2 (TextV  (None, 30)               0         
 ectorization)                                                   
                                                                 
 embedding_6 (Embedding)     (None, 30, 300)           9000600   
                                                                 
 bidirectional_6 (Bidirectio  (None, 30, 128)          186880    
 nal)                                                            
                                                                 
 bidirectional_7 (Bidirectio  (None, 128)              74496     
 nal)                                                            
                                                           

In [ ]:
es = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=70, restore_best_weights=True)

batch_size = 768
epochs = 3
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), callbacks=[es], epochs=epochs, batch_size=batch_size)

Epoch 1/3
1055/1055 [==============================] - 266s 248ms/step - loss: 0.4665 - accuracy: 0.7774 - val_loss: 0.4219 - val_accuracy: 0.8042
Epoch 2/3
1055/1055 [==============================] - 259s 246ms/step - loss: 0.4151 - accuracy: 0.8105 - val_loss: 0.4088 - val_accuracy: 0.8113
Epoch 3/3
1055/1055 [==============================] - 258s 244ms/step - loss: 0.4002 - accuracy: 0.8190 - val_loss: 0.3956 - val_accuracy: 0.8191


In [ ]:
model.evaluate(x_test,y_test)

313/313 [==============================] - 6s 20ms/step - loss: 0.3968 - accuracy: 0.8198


[0.3967535197734833, 0.8198000192642212]

In [ ]:
y_pred=model.predict(x_test)

accuracy_sc = accuracy_score(y_pred=y_pred.round(),y_true=y_test)*100
f1_sc = f1_score(y_pred=y_pred.round(),y_true=y_test)

print("Accuracy score is {}% ".format(accuracy_sc))
print("f1-score is {}% ".format(f1_sc))
print(classification_report(y_pred=y_pred.round(),y_true=y_test))

Accuracy score is 81.98% 
f1-score is 0.8247763516141579% 
              precision    recall  f1-score   support

           0       0.84      0.79      0.81      5000
           1       0.80      0.85      0.82      5000

    accuracy                           0.82     10000
   macro avg       0.82      0.82      0.82     10000
weighted avg       0.82      0.82      0.82     10000



# 7 - 8 Vectorizer parameters

In [ ]:
embedding_dim = 300 # Dimension of embedded representation 
vocab_size = 100000 # Number of unique tokens in vocabulary
sequence_length = 30 # Output dimension after vectorizing

vect_layer = TextVectorization(max_tokens=vocab_size, output_mode='int', output_sequence_length=sequence_length)
vect_layer.adapt(x_train)

In [ ]:
voc = vect_layer.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))

In [ ]:
num_tokens = len(voc) + 2
hits = 0
misses = 0


embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

Converted 54599 words (45401 misses)


In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.10, random_state=69, stratify = y_train)

## 7.Experiment

In [ ]:
input_layer = keras.layers.Input(shape=(1,), dtype=tf_string)
x_v = vect_layer(input_layer)
emb = keras.layers.Embedding(vocab_size, embedding_dim)(x_v)
x = LSTM(64, activation='relu', return_sequences=True)(emb)
x = LSTM(64, activation='relu', return_sequences=True)(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(64, 'relu')(x)
x = keras.layers.Dense(32, 'relu')(x)
x = keras.layers.Dropout(0.2)(x)
output_layer = keras.layers.Dense(1, 'sigmoid')(x)

model = keras.Model(input_layer, output_layer)
model.summary()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

model.compile(optimizer=optimizer, loss=keras.losses.BinaryCrossentropy(), metrics=['accuracy'])

Model: "model_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_9 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_6 (TextV  (None, 30)               0         
 ectorization)                                                   
                                                                 
 embedding_8 (Embedding)     (None, 30, 300)           30000000  
                                                                 
 lstm_9 (LSTM)               (None, 30, 64)            93440     
                                                                 
 lstm_10 (LSTM)              (None, 30, 64)            33024     
                                                                 
 flatten_8 (Flatten)         (None, 1920)              0         
                                                           

In [ ]:
es = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=70, restore_best_weights=True)

batch_size = 768
epochs = 2
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), callbacks=[es], epochs=epochs, batch_size=batch_size)

Epoch 1/2
1055/1055 [==============================] - 150s 140ms/step - loss: 0.4295 - accuracy: 0.8005 - val_loss: 0.3915 - val_accuracy: 0.8229
Epoch 2/2
1055/1055 [==============================] - 145s 138ms/step - loss: 0.3509 - accuracy: 0.8451 - val_loss: 0.4013 - val_accuracy: 0.8202


In [ ]:
model.evaluate(x_test,y_test)

313/313 [==============================] - 4s 11ms/step - loss: 0.4006 - accuracy: 0.8198


[0.40057894587516785, 0.8198000192642212]

In [ ]:
y_pred=model.predict(x_test)

accuracy_sc = accuracy_score(y_pred=y_pred.round(),y_true=y_test)*100
f1_sc = f1_score(y_pred=y_pred.round(),y_true=y_test)

print("Accuracy score is {}% ".format(accuracy_sc))
print("f1-score is {}% ".format(f1_sc))
print(classification_report(y_pred=y_pred.round(),y_true=y_test))

Accuracy score is 81.98% 
f1-score is 0.8121742755889098% 
              precision    recall  f1-score   support

           0       0.80      0.86      0.83      5000
           1       0.85      0.78      0.81      5000

    accuracy                           0.82     10000
   macro avg       0.82      0.82      0.82     10000
weighted avg       0.82      0.82      0.82     10000



## 8.Experiment

In [ ]:
input_layer = keras.layers.Input(shape=(1,), dtype=tf_string)
x_v = vect_layer(input_layer)
emb = keras.layers.Embedding(vocab_size, embedding_dim)(x_v)
x = Bidirectional(LSTM(128, activation='relu', return_sequences=True))(emb)
x = Bidirectional(LSTM(64, activation='relu', return_sequences=True))(x)
x = Bidirectional(GRU(64, activation='relu', return_sequences=True))(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(64, 'relu')(x)
x = keras.layers.Dense(32, 'relu')(x)
x = keras.layers.Dropout(0.2)(x)
output_layer = keras.layers.Dense(1, 'sigmoid')(x)

model = keras.Model(input_layer, output_layer)
model.summary()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

model.compile(optimizer=optimizer, loss=keras.losses.BinaryCrossentropy(), metrics=['accuracy'])

Model: "model_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_10 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization_6 (TextV  (None, 30)               0         
 ectorization)                                                   
                                                                 
 embedding_9 (Embedding)     (None, 30, 300)           30000000  
                                                                 
 bidirectional_8 (Bidirectio  (None, 30, 256)          439296    
 nal)                                                            
                                                                 
 bidirectional_9 (Bidirectio  (None, 30, 128)          164352    
 nal)                                                            
                                                           

In [ ]:
es = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=70, restore_best_weights=True)

batch_size = 768
epochs = 2
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), callbacks=[es], epochs=epochs, batch_size=batch_size)

Epoch 1/2
1055/1055 [==============================] - 394s 365ms/step - loss: 0.4279 - accuracy: 0.8007 - val_loss: 0.3912 - val_accuracy: 0.8231
Epoch 2/2
1055/1055 [==============================] - 385s 365ms/step - loss: 0.3473 - accuracy: 0.8473 - val_loss: 0.3877 - val_accuracy: 0.8252


In [ ]:
model.evaluate(x_test,y_test)

313/313 [==============================] - 9s 29ms/step - loss: 0.3860 - accuracy: 0.8236


[0.3860154151916504, 0.8235999941825867]

In [ ]:
y_pred=model.predict(x_test)

accuracy_sc = accuracy_score(y_pred=y_pred.round(),y_true=y_test)*100
f1_sc = f1_score(y_pred=y_pred.round(),y_true=y_test)

print("Accuracy score is {}% ".format(accuracy_sc))
print("f1-score is {}% ".format(f1_sc))
print(classification_report(y_pred=y_pred.round(),y_true=y_test))

Accuracy score is 82.36% 
f1-score is 0.8238466147393648% 
              precision    recall  f1-score   support

           0       0.82      0.82      0.82      5000
           1       0.82      0.82      0.82      5000

    accuracy                           0.82     10000
   macro avg       0.82      0.82      0.82     10000
weighted avg       0.82      0.82      0.82     10000



# Save model

In [ ]:
model.save('FT_forth_done')

INFO:tensorflow:Assets written to: lstm_forth_done/assets


INFO:tensorflow:Assets written to: lstm_forth_done/assets


In [ ]:
!zip -r /content/FTforthD.zip /content/FT_forth_done/

  adding: content/lstm_forth_done/ (stored 0%)
  adding: content/lstm_forth_done/saved_model.pb (deflated 78%)
  adding: content/lstm_forth_done/assets/ (stored 0%)
  adding: content/lstm_forth_done/variables/ (stored 0%)
  adding: content/lstm_forth_done/variables/variables.index (deflated 66%)
  adding: content/lstm_forth_done/variables/variables.data-00000-of-00001 (deflated 7%)
  adding: content/lstm_forth_done/keras_metadata.pb (deflated 90%)
